In [ ]:
import kagglehub
import shutil
import os

# Download dataset (cached by kagglehub)
cache_path = kagglehub.dataset_download("omkargurav/face-mask-dataset")
print("Downloaded to cache:", cache_path)

# Your LOCAL project directory
target_dir = os.path.join(os.getcwd(), "face_mask_dataset")

# Create directory if not exists
os.makedirs(target_dir, exist_ok=True)

# Copy dataset from cache to project folder
shutil.copytree(cache_path, target_dir, dirs_exist_ok=True)

print("Dataset copied to:", target_dir)


In [ ]:
!ls face_mask_dataset/data


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model


In [ ]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
DATA_DIR = "face_mask_dataset/data"



In [ ]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,   # 80% train, 20% val
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

train_data = datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="training"
)

val_data = datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="validation"
)


In [ ]:
base_model = MobileNetV2(
    input_shape=(224,224,3),
    include_top=False,
    weights="imagenet"
)

base_model.trainable = False  # IMPORTANT for fast training

x = base_model.output
x = GlobalAveragePooling2D()(x)
output = Dense(1, activation="sigmoid")(x)

model = Model(inputs=base_model.input, outputs=output)


In [ ]:
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)


In [ ]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=5
)


In [ ]:
model.save("face_mask_detection_model.h5")


In [ ]:
# ================================
# Face Mask Detection - URL Input
# ================================

import numpy as np
import matplotlib.pyplot as plt
import requests
from io import BytesIO
from PIL import Image
from tensorflow.keras.preprocessing.image import img_to_array

# ---- CONFIG ----
IMG_SIZE = (224, 224)   # same size used during training

# ---- USER INPUT ----
image_url = input("👉 Paste image URL here: ")

# ---- LOAD IMAGE FROM URL ----
response = requests.get(image_url)
img = Image.open(BytesIO(response.content)).convert("RGB")
img = img.resize(IMG_SIZE)

# ---- PREPROCESS ----
img_array = img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# ---- PREDICT ----
pred = model.predict(img_array)[0][0]

if pred < 0.5:
    label = "With Mask 😷"
    confidence = (1 - pred) * 100
else:
    label = "Without Mask ❌"
    confidence = pred * 100

# ---- DISPLAY RESULT ----
plt.imshow(img)
plt.axis("off")
plt.title(f"{label} | Confidence: {confidence:.2f}%")
plt.show()
